In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold

from util import gini_normalized
from parameters import parameters, batch_size, epochs, layers, activation_functions, loss, alpha
from preprocessing import preproc

In [ ]:
# Importing the train dataset
dataset_train = pd.read_csv('train.csv')

# Importing the test dataset
dataset_test = pd.read_csv('test.csv')
# preprocessing train dataset
X_train, y_train, scaler = preproc(dataset_train, 'train', oneHot=True, scale=True)

# preprocessing test dataset
X_test, y_test = preproc(dataset_test, 'test', oneHot=True, scale=True, scaler=scaler)

In [ ]:
# Now let's make the Classifier!
# Fitting Random Forest Classification to the Training set

class_weight = {0: 1., 1: alpha}
K = 5
kf = KFold(n_splits=K, random_state=42, shuffle=True)

In [ ]:
i=0
results = []
for train_index, test_index in kf.split(X_train):
    train_x, train_y = X_train[train_index], y_train[train_index]
    eval_x, eval_y = X_train[test_index], y_train[test_index]
    classifier = RandomForestClassifier(n_estimators=10, criterion = 'gini', random_state = 1, max_features='auto', class_weight=class_weight)
    classifier.fit(train_x, train_y)
    res_eval = classifier.predict(eval_x)
    res = classifier.predict(X_test)
    results.append(res)
    print('gini_eval', i)
    gini_score = gini_normalized(eval_y, res_eval)
    print(gini_score)
    i+=1
